#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

In [2]:
import pandas as pd
import requests

In [5]:
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape

(27028, 69)


(27028, 69)

In [9]:
total_registros = sinasc.shape[0]
registros_unicos = sinasc.drop_duplicates().shape[0]
print(f"Total de registros: {total_registros}")
print(f"Total de registros não duplicados: {registros_unicos}")

Total de registros: 27028
Total de registros não duplicados: 27028


In [10]:
if total_registros == registros_unicos:
    print("Não há duplicados.")
else:
    print(f"Há {total_registros - registros_unicos} registros duplicados.")

Não há duplicados.


2. Conte o número de valores *missing* por variável.  

In [13]:
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

In [18]:
columns_of_interest = [
    'LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 
    'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'
]
sinasc_selected = sinasc[columns_of_interest]

sinasc_selected.isna().sum() 

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.

In [17]:
sinasc_filtered = sinasc_selected.dropna(subset=['APGAR5'])
remaining_rows = len(sinasc_filtered)
missing_values_after_filter = sinasc_filtered.isna().sum()

remaining_rows, missing_values_after_filter

(26925,
 LOCNASC          0
 IDADEMAE         0
 ESTCIVMAE      315
 ESCMAE         310
 QTDFILVIVO    1566
 GESTACAO      1216
 GRAVIDEZ        76
 CONSULTAS        0
 APGAR5           0
 dtype: int64)

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>

In [24]:
sinasc_filtered['ESTCIVMAE'] = sinasc_filtered['ESTCIVMAE'].fillna(9)
sinasc_filtered['CONSULTAS'] = sinasc_filtered['CONSULTAS'].fillna(9)

print("missing após a substituição:")
print(sinasc_filtered[['ESTCIVMAE', 'CONSULTAS']].isna().sum())

missing após a substituição:
ESTCIVMAE    0
CONSULTAS    0
dtype: int64


6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  

In [23]:
sinasc_filtered['QTDFILVIVO'] = sinasc_filtered['QTDFILVIVO'].fillna(0)

print("missing após a substituição em QTDFILVIVO:")
print(sinasc_filtered['QTDFILVIVO'].isna().sum())

missing após a substituição em QTDFILVIVO:
0


7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui. 

In [29]:
sinasc_filtered['LOCNASC'] = sinasc_filtered['LOCNASC'].fillna(sinasc_filtered['LOCNASC'].mode()[0])

sinasc_filtered['IDADEMAE'] = sinasc_filtered['IDADEMAE'].fillna(sinasc_filtered['IDADEMAE'].mean())

sinasc_filtered['ESTCIVMAE'] = sinasc_filtered['ESTCIVMAE'].fillna(sinasc_filtered['ESTCIVMAE'].mode()[0])

sinasc_filtered['ESCMAE'] = sinasc_filtered['ESCMAE'].fillna(sinasc_filtered['ESCMAE'].mode()[0])

sinasc_filtered['GESTACAO'] = pd.to_numeric(sinasc_filtered['GESTACAO'], errors='coerce')

sinasc_filtered['GESTACAO'] = sinasc_filtered['GESTACAO'].fillna(sinasc_filtered['GESTACAO'].mean())

sinasc_filtered['GRAVIDEZ'] = sinasc_filtered['GRAVIDEZ'].fillna(sinasc_filtered['GRAVIDEZ'].mode()[0])

sinasc_filtered['CONSULTAS'] = sinasc_filtered['CONSULTAS'].fillna(0)


8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

In [35]:
def categorizar_apgar(score):
    if score >= 8:
        return 'Normal'
    elif score >= 6:
        return 'Asfixia leve'
    elif score >= 4:
        return 'Asfixia moderada'
    else:
        return 'Asfixia severa'


sinasc['Apgar_categoria'] = sinasc['APGAR5'].apply(categorizar_apgar)
frequencias = sinasc['Apgar_categoria'].value_counts()
print(frequencias)



Apgar_categoria
Normal              26463
Asfixia leve          320
Asfixia severa        177
Asfixia moderada       68
Name: count, dtype: int64


9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [36]:
sinasc.rename(columns=lambda x: x.strip().replace(' ', '_').replace('.', '').replace('-', '_').lower(), inplace=True)

print(sinasc.columns)

Index(['origem', 'codestab', 'codmunnasc', 'locnasc', 'idademae', 'estcivmae',
       'escmae', 'codocupmae', 'qtdfilvivo', 'qtdfilmort', 'codmunres',
       'gestacao', 'gravidez', 'parto', 'consultas', 'dtnasc', 'horanasc',
       'sexo', 'apgar1', 'apgar5', 'racacor', 'peso', 'idanomal', 'dtcadastro',
       'codanomal', 'numerolote', 'versaosist', 'dtrecebim', 'difdata',
       'dtrecoriga', 'naturalmae', 'codmunnatu', 'codufnatu', 'escmae2010',
       'seriescmae', 'dtnascmae', 'racacormae', 'qtdgestant', 'qtdpartnor',
       'qtdpartces', 'idadepai', 'dtultmenst', 'semagestac', 'tpmetestim',
       'consprenat', 'mesprenat', 'tpapresent', 'sttrabpart', 'stcesparto',
       'tpnascassi', 'tpfuncresp', 'tpdocresp', 'dtdeclarac', 'escmaeagr1',
       'stdnepidem', 'stdnnova', 'codpaisres', 'tprobson', 'paridade',
       'kotelchuck', 'contador', 'munresstatus', 'munrestipo', 'munresnome',
       'munresuf', 'munreslat', 'munreslon', 'munresalt', 'munresarea',
       'apgar_categoria